### [Repo](https://github.com/theedgepredictor/espn-api-orm)

Project is started

POC down and research below

# ESPN API ORM

Object Relational Mapper for ESPN API. 

POC:
- Scoreboard API

Additional:
- Odds
- Sport
- Team
- Season
- Boxscore
- Play by Play
- Player


## Simple

Game: 20230404_uconn_sandiegostate
- Info: Last years final
- Game_id: 401522202
- Home_id: 41
- Away_id: 21


In [1]:
import time

import requests
from enum import Enum

######################################
# Sport Consts
######################################

class ESPNSportTypes(Enum):
    COLLEGE_BASKETBALL = 'basketball/mens-college-basketball'
    COLLEGE_FOOTBALL = 'football/college-football'
    COLLEGE_BASEBALL = 'baseball/college-baseball'
    COLLEGE_HOCKEY = 'hockey/mens-college-hockey'
    COLLEGE_LACROSSE = 'lacrosse/mens-college-lacrosse'
    NBA = 'basketball/nba'
    NFL = 'football/nfl'
    MLB = 'baseball/mlb'
    NHL = 'hockey/nhl'
    PLL = 'lacrosse/pll'

class ESPNSportSeasonTypes(Enum):
    PRE = 1
    REG = 2
    POST = 3
    OFF = 4


##################################
# Scoreboard Classes
##################################

from typing import List, Optional, Any
from pydantic import BaseModel
import datetime

## Leagues

class Logo(BaseModel):
    href: str
    width: int
    height: int
    alt: str
    rel: List[str]
    lastUpdated: datetime.datetime

class League(BaseModel):
    id: str
    uid: str
    name: str
    abbreviation: str
    midsizeName: Optional[str] = None
    slug: str
    season: 'Season'
    logos: List[Logo]
    calendarType: str
    calendarIsWhitelist: bool
    calendarStartDate: datetime.datetime
    calendarEndDate: datetime.datetime
    calendar: List[Any]

    class Season(BaseModel):
        year: int
        startDate: datetime.datetime
        endDate: datetime.datetime
        displayName: str
        type: 'SeasonType'  

        class SeasonType(BaseModel):
            id: int
            type: int
            name: str
            abbreviation: str

## Day
class TeamRef(BaseModel):
    id: int

class VenueRef(BaseModel):
    id: int

class Day(BaseModel):
    date: str

class EventDate(BaseModel):
    date: datetime.datetime
    seasonType: int
    
## Events
    
class Link(BaseModel):
    rel: List[str]
    href: str
    text: Optional[str] = None
    isExternal: Optional[bool] = False
    isPremium: Optional[bool] = False

class Athlete(BaseModel):
    id: str
    fullName: str
    displayName: str
    shortName: str
    links: List['Link']
    headshot: Optional[str] = None
    jersey: Optional[str] = None
    position: 'Position'
    team: 'TeamRef'
    active: bool

    class Position(BaseModel):
        abbreviation: str

class Address(BaseModel):
    city: str
    state: Optional[str] = None

class CuratedRank(BaseModel):
    current: int

class Team(BaseModel):
    id: int
    uid: str
    location: str
    name: Optional[str] = None
    abbreviation: str
    displayName: str
    shortDisplayName: str
    color: Optional[str] = None
    alternateColor: Optional[str] = None
    isActive: bool
    venue: Optional['VenueRef'] = None
    links: List['Link']
    logo: Optional[str] = None
    conferenceId: Optional[int] = None

class Competitor(BaseModel):
    id: str
    uid: str
    type: str
    order: int
    homeAway: str
    winner: Optional[bool] = None
    team: 'Team'
    score: str
    linescores: List['Linescore'] = []
    statistics: List['Statistic'] = []
    leaders: List['LeaderGroup'] = []
    curatedRank: Optional[CuratedRank] = None
    records: List['Record'] = []

    class Linescore(BaseModel):
        value: float

    class Statistic(BaseModel):
        name: str
        abbreviation: str
        displayValue: str

    class Record(BaseModel):
        name: str
        abbreviation: Optional[str] = None
        type: str
        summary: str

    class LeaderGroup(BaseModel):
        name: str
        displayName: str
        shortDisplayName: str
        abbreviation: str
        leaders: List['Leader'] = None

        class Leader(BaseModel):
            displayValue: str
            value: float
            athlete: 'Athlete'
            team: 'TeamRef'

class Competition(BaseModel):
    id: str
    uid: str
    date: datetime.datetime
    attendance: Optional[int] = None
    type: Optional['Type'] = None
    timeValid: bool
    neutralSite: bool
    conferenceCompetition: Optional[bool] = None
    playByPlayAvailable: bool
    recent: bool
    venue: Optional['Venue'] = None
    competitors: List[Competitor]

    class Venue(BaseModel):
        id: str
        fullName: str
        address: 'Address'
        capacity: Optional[int] = None
        indoor: Optional[bool] = None

    class Type(BaseModel):
        id: str
        abbreviation: str
    
class Event(BaseModel):
    id: str
    uid: str
    date: datetime.datetime
    name: str
    shortName: str
    season: 'Season'
    competitions: List[Competition]
    links: List['Link']
    status: 'Status'

    class Status(BaseModel):
        clock: Optional[float] = None
        displayClock: Optional[str] = None
        period: int
        type: 'Type'

        class Type(BaseModel):
            id: str
            name: str
            state: str
            completed: bool
            description: str
            detail: str
            shortDetail: str

    class Season(BaseModel):
        year: int
        type: int
        slug: str


class Scoreboard(BaseModel):
    leagues: List[League]
    groups: List[str] = None
    day: Optional[Day] = None
    eventsDate: Optional[EventDate] = None
    events: List[Event]


class ESPNBaseAPI:
    """
    ESPNBaseAPI class for making API requests to ESPN's sports data endpoints.

    Attributes:
        _base_url (str): The base URL for ESPN's public API.
        _core_url (str): The base URL for ESPN's core API.

    Methods:
        api_request(url: str, retry_count: int = 0) -> dict or None:
            Makes an API request to the specified URL.

            Args:
                url (str): The complete URL for the API request.
                retry_count (int): The number of times to retry the request in case of failure. Default is 0.

            Returns:
                dict or None: The JSON response from the API, or None if the request was unsuccessful.
                If the response indicates a 404 status code or an error, None is returned.

            Raises:
                Exception: Raises an exception if the request encounters an error after multiple retries.
                This is typically used when the request limit is exceeded (error code 2502).
    """

    def __init__(self):
        """
        Initializes an instance of the ESPNBaseAPI class.

        Attributes:
            _base_url (str): The base URL for ESPN's public API.
            _core_url (str): The base URL for ESPN's core API.
        """
        self._base_url = 'https://site.api.espn.com/apis/site/v2/sports'
        self._core_url = 'https://sports.core.api.espn.com/v2/sports'

    def api_request(self, url: str, retry_count: int = 0) -> dict or None:
        """
        Makes an API request to the specified URL.

        Args:
            url (str): The complete URL for the API request.
            retry_count (int): The number of times to retry the request in case of failure. Default is 0.

        Returns:
            dict or None: The JSON response from the API, or None if the request was unsuccessful.
            If the response indicates a 404 status code or an error, None is returned.

        Raises:
            Exception: Raises an exception if the request encounters an error after multiple retries.
            This is typically used when the request limit is exceeded (error code 2502).
        """
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
            }
            resp = requests.get(url=url, headers=headers)
            if resp.status_code == 404:
                return None
            res = resp.json()
            if 'error' in res:
                if res['error']['code'] == 404:  # No data
                    return None
            if 'code' in res:
                if res['code'] == 2502:
                    raise Exception('Flooded')  # Too many requests
                if res['code'] == 400:  # Data cant be found (wrong endpoint/wrong request)
                    return None
            return res
        except Exception as e:
            if retry_count >= 3:
                raise e
            time.sleep(5)
            print(f'URL error for {url}')
            self.api_request(url, retry_count=retry_count + 1)

import numpy as np

class ESPNEventsAPI(ESPNBaseAPI):
    """
    ESPN Events API for retrieving sports events information.

    Attributes:
        SCHEMA (dict): Dictionary defining the data schema for events.

    Methods:
        get_scoreboard(sport, dates, limit=1000, groups=None): Retrieve scoreboard data for a specific sport.
        get_events(sport, dates, limit=1000, groups=None): Retrieve events data for a specific sport.
        get_events_for_elo(sport, dates, limit=1000, groups=None): Retrieve events data suitable for Elo calculations.
        _collect_elo_payload(event, name_type='shortDisplayName'): Collect Elo payload for a given event.
        _team_name_validator(name): Validate and filter team names.

    """
    def __init__(self):
        """
        Initialize ESPNEventsAPI.
        """
        super().__init__()
        self.SCHEMA = {
            'id':np.int64,
            'season':np.int32,
            'is_postseason': np.int8,
            'tournament_id':'Int32',
            'is_finished': np.int8,
            'neutral_site': np.int8,
            'home_team_id':np.int32,
            'home_team_score':'Int32',
            'away_team_id':np.int32,
            'away_team_score':'Int32',
        }

    def get_scoreboard(self, sport: ESPNSportTypes, dates, limit=1000, groups=None) -> Optional[Scoreboard]:
        """
        Retrieve scoreboard data for a specific sport.

        Args:
            sport (ESPNSportTypes): Type of sport.
            dates: Dates for events.
            limit (int): Limit of events to retrieve.
            groups: Groups for events.

        Returns:
            dict: API response containing scoreboard data.
        """
        url = f"{self._base_url}/{sport.value}/scoreboard?dates={dates}&limit={limit}"
        if groups is not None:
            url=f"{url}&groups={groups}"
        return Scoreboard(**self.api_request(url))

    def get_events(self, sport: ESPNSportTypes, dates, limit=1000, groups=None) -> List[Event]:
        """
        Retrieve events data for a specific sport.

        Args:
            sport (ESPNSportTypes): Type of sport.
            dates: Dates for events.
            limit (int): Limit of events to retrieve.
            groups: Groups for events.

        Returns:
            list: List of events data.
        """
        res = self.get_scoreboard(sport, dates, limit, groups)
        if res is None:
            return []
        return res.events




## Schema (Scoreboard)

Issue is Scoreboard is a aggregate view of many different payloads. Its great for getting an overview of an event but some keys in the schema are only 'sometimes' available but do exist elsewhere (odds)

base: https://site.api.espn.com/apis/site/v2/sports/{SPORT}/scoreboard?dates={CALENDAR_DATE}&limit=1000
res:
- leagues List:
    - ['id', 'uid', 'name', 'abbreviation', 'midsizeName', 'slug', 'calendarType', 'calendarIsWhitelist', 'calendarStartDate', 'calendarEndDate']
    - season: ['year', 'startDate', 'endDate', 'displayName']
        - type: ['id', 'type', 'name', 'abbreviation']
    - logos: List[Logos]
    - calendar: List[CALENDAR_DATE]
- groups: List[group_id]
- day: date
- events List:
    - ['id', 'uid', 'date', 'name', 'shortName', 'season', 'competitions',  'status']
    - links: List[Link]
    - status: ['clock', 'displayClock', 'period']
        - type: ['id', 'name', 'state', 'completed', 'description', 'detail', 'shortDetail']
    - competitions List: ['id', 'uid', 'date', 'attendance','timeValid', 'neutralSite', 'conferenceCompetition', 'playByPlayAvailable', 'recent', 'tournamentId', 'format', 'startDate']
        - type: ['id', 'abbreviation']
        - venue: ['id', 'fullName', 'address', 'capacity', 'indoor']
        - notes: List[NOTE]
        - status: ['clock', 'displayClock', 'period', 'type']
        - broadcasts: List[Broadcasts]
        - geoBroadcasts: List[GeoBroadcasts]
        - headlines: List[Headline]
        - competitors List: ['id', 'uid', 'type', 'order', 'homeAway', 'winner', 'score']
            - linescores: List[LineScores]
            - statistics: List[Statistics]
            - leaders: List[Leaders]
            - curatedRank
            - records: List[TeamRecordSplits]
            - team: ['id', 'uid', 'location', 'name', 'abbreviation', 'displayName', 'shortDisplayName', 'color', 'alternateColor', 'isActive', 'venue', 'links', 'logo', 'conferenceId']
- eventsDate

## Endpoints to Link



In [2]:
sport = ESPNSportTypes.COLLEGE_BASKETBALL
sport_str = sport.value
gameid = 401522202
home_id = 41
away_id = 21
espn_core_name = sport.value.split('/')[0] + '/leagues/' + sport.value.split('/')[1]
base_api = ESPNBaseAPI()

#res = base_api.api_request(f"{base_api._base_url}/{sport_str}/scoreboard?dates=20230403&limit=10")

In [7]:
reg_res = base_api.api_request(f"{base_api._core_url}/{espn_core_name}/seasons/{2022}/types/{ESPNSportSeasonTypes.REG.value}")

In [17]:
reg_res = base_api.api_request(f"{base_api._core_url}/{espn_core_name}/seasons/{2024}/teams/2")
reg_res['venue']

{'$ref': 'http://sports.core.api.espn.com/v2/sports/basketball/leagues/mens-college-basketball/venues/1953?lang=en&region=us',
 'id': '1953',
 'fullName': 'Neville Arena',
 'address': {'city': 'Auburn', 'state': 'AL'},
 'capacity': 0,
 'grass': False,
 'indoor': True,
 'images': [{'href': 'https://a.espncdn.com/i/venues/mens-college-basketball/day/1953.jpg',
   'width': 2000,
   'height': 1125,
   'alt': '',
   'rel': ['full', 'day']}]}

In [12]:
from pydantic import BaseModel, Field

class ESPNSportTypes(Enum):
    AUSTRALIAN_FOOTBALL = 'australian-football'
    BASEBALL = 'baseball'
    BASKETBALL = 'basketball'
    CRICKET = 'cricket'
    FIELD_HOCKEY = 'field-hockey'
    FOOTBALL = 'football'
    GOLF = 'golf'
    HOCKEY = 'hockey'
    LACROSSE = 'lacrosse'
    MMA = 'mma'
    RACING = 'racing'
    RUGBY = 'rugby'
    RUGBY_LEAGUE = 'rugby-league'
    SOCCER = 'soccer'
    TENNIS = 'tennis'
    VOLLEYBALL = 'volleyball'
    WATER_POLO = 'water-polo'

class Ref(BaseModel):
    ref: str = Field(..., alias='$ref')


class Sports(BaseModel):
    count: int
    pageIndex: int
    pageSize: int
    pageCount: int
    items: List[Ref]

class Sport(BaseModel):
    ref: str = Field(..., alias='$ref')
    id: int
    guid: Optional[str] = None
    uid: str
    name: str
    displayName: str
    slug: str
    leagues: Ref
    logos: List[Logo]

class SportLeague(BaseModel):
    count: int
    pageIndex: int
    pageSize: int
    pageCount: int
    items: List[Ref]
class ESPNSportAPI(ESPNBaseAPI):
    """
    ESPN Sport API for retrieving sport information.

    Attributes:

    Methods:
    """

    def __init__(self):
        """
        Initialize ESPNSportAPI.
        """
        super().__init__()

    def get_sports(self, return_values = True):
        res = Sports(**self.api_request(f"{self._core_url}"))
        if not return_values:
            return res
        return [sport.ref.replace(f"{self._core_url.replace('https','http')}/",'').split('?')[0] for sport in res.items]

    def get_sport(self, sport: ESPNSportTypes):
        return Sport(**self.api_request(f"{self._core_url}/{sport.value}"))

    def get_leagues_for_sport(self, sport: ESPNSportTypes, return_values = True):
        res = SportLeague(**self.api_request(f"{self._core_url}/{sport.value}/leagues"))
        if not return_values:
            return res
        return [league.ref.replace(f"{self._core_url.replace('https','http')}/{sport.value}/leagues/",'').split('?')[0] for league in res.items]
        
class ESPNLeagueAPI(ESPNBaseAPI):
    """
    ESPN League API for retrieving sport information about a specific league.

    Attributes:

    Methods:
    """

    def __init__(self, sport: ESPNSportTypes, league: str):
        """
        Initialize ESPNSportAPI.
        """
        super().__init__()
        self.sport: ESPNSportTypes = sport
        self.league = league

    def get_league(self):
        return self.api_request(f"{self._core_url}/{self.sport.value}/leagues/{self.league}")
    
class SeasonType(BaseModel):
    ref: str = Field(..., alias='$ref')
    id: int
    type: int
    name: str
    abbreviation: str
    year: int
    startDate: datetime.datetime
    endDate: datetime.datetime
    hasGroups: bool
    hasStandings: bool
    hasLegs: bool
    groups: Optional[Ref] = None
    week: Optional['Week'] = None
    weeks: Optional[Ref] = None
    leaders: Optional[Ref] = None
    slug: str

    class Week(BaseModel):
        ref: str = Field(..., alias='$ref')
        number: int
        startDate: datetime.datetime
        endDate: datetime.datetime
        text: str
        rankings: Ref


class Season(BaseModel):
    ref: str = Field(..., alias='$ref')
    year: int
    startDate: datetime.datetime
    endDate: datetime.datetime
    displayName: str
    type: SeasonType
    types: 'Types'
    rankings: Ref
    powerIndexes: Ref
    powerIndexLeaders: Ref
    athletes: Ref
    futures: Ref
    leaders: Ref

    class Types(BaseModel):
        ref: str = Field(..., alias='$ref')
        count: int
        pageIndex: int
        pageSize: int
        pageCount: int
        items: List[SeasonType]

In [13]:
api = ESPNLeagueAPI(ESPNSportTypes.BASKETBALL, 'mens-college-basketball')

Season(ref='http://sports.core.api.espn.com/v2/sports/basketball/leagues/mens-college-basketball/seasons/2024?lang=en&region=us', year=2024, startDate=datetime.datetime(2023, 7, 13, 7, 0, tzinfo=TzInfo(UTC)), endDate=datetime.datetime(2024, 4, 10, 6, 59, tzinfo=TzInfo(UTC)), displayName='2023-24', type=SeasonType(ref='http://sports.core.api.espn.com/v2/sports/basketball/leagues/mens-college-basketball/seasons/2024/types/2?lang=en&region=us', id=2, type=2, name='Regular Season', abbreviation='reg', year=2024, startDate=datetime.datetime(2023, 11, 6, 8, 0, tzinfo=TzInfo(UTC)), endDate=datetime.datetime(2024, 3, 18, 6, 59, tzinfo=TzInfo(UTC)), hasGroups=False, hasStandings=True, hasLegs=False, groups=Ref(ref='http://sports.core.api.espn.com/v2/sports/basketball/leagues/mens-college-basketball/seasons/2024/types/2/groups?lang=en&region=us'), week=Week(ref='http://sports.core.api.espn.com/v2/sports/basketball/leagues/mens-college-basketball/seasons/2024/types/2/weeks/17?lang=en&region=us', 

In [7]:
api.get_calendar_for_season(sport,2024)

AttributeError: 'ESPNLeagueAPI' object has no attribute 'get_calendar_for_season'

In [ ]:
from enum import Enum

## All Sport/League Pairs 
class ESPNSportTypes(Enum):
    BASEBALL_COLLEGE_BASEBALL = 'baseball/college-baseball'
    BASEBALL_COLLEGE_SOFTBALL = 'baseball/college-softball'
    BASEBALL_MLB = 'baseball/mlb'
    BASKETBALL_MENS_COLLEGE_BASKETBALL = 'basketball/mens-college-basketball'
    BASKETBALL_MENS_OLYMPICS_BASKETBALL = 'basketball/mens-olympics-basketball'
    BASKETBALL_NBA = 'basketball/nba'
    BASKETBALL_WNBA = 'basketball/wnba'
    BASKETBALL_WOMENS_COLLEGE_BASKETBALL = 'basketball/womens-college-basketball'
    FIELD_HOCKEY_WOMENS_COLLEGE_FIELD_HOCKEY = 'field-hockey/womens-college-field-hockey'
    FOOTBALL_COLLEGE_FOOTBALL = 'football/college-football'
    FOOTBALL_NFL = 'football/nfl'
    GOLF_LIV = 'golf/liv'
    GOLF_LPGA = 'golf/lpga'
    GOLF_PGA = 'golf/pga'
    HOCKEY_MENS_COLLEGE_HOCKEY = 'hockey/mens-college-hockey'
    HOCKEY_NHL = 'hockey/nhl'
    HOCKEY_WOMENS_COLLEGE_HOCKEY = 'hockey/womens-college-hockey'
    LACROSSE_MENS_COLLEGE_LACROSSE = 'lacrosse/mens-college-lacrosse'
    LACROSSE_PLL = 'lacrosse/pll'
    LACROSSE_WOMENS_COLLEGE_LACROSSE = 'lacrosse/womens-college-lacrosse'
    MMA_UFC = 'mma/ufc'
    SOCCER_UEFA_CHAMPIONS = 'soccer/uefa.champions'
    SOCCER_ENG_1 = 'soccer/eng.1'
    SOCCER_ESP_1 = 'soccer/esp.1'
    SOCCER_GER_1 = 'soccer/ger.1'
    SOCCER_USA_1 = 'soccer/usa.1'
    SOCCER_ITA_1 = 'soccer/ita.1'
    SOCCER_FRA_1 = 'soccer/fra.1'
    SOCCER_ENG_2 = 'soccer/eng.2'
    SOCCER_NED_1 = 'soccer/ned.1'
    SOCCER_POR_1 = 'soccer/por.1'
    VOLLEYBALL_MENS_COLLEGE_VOLLEYBALL = 'volleyball/mens-college-volleyball'
    VOLLEYBALL_WOMENS_COLLEGE_VOLLEYBALL = 'volleyball/womens-college-volleyball'
    WATER_POLO_MENS_COLLEGE_WATER_POLO = 'water-polo/mens-college-water-polo'
    WATER_POLO_WOMENS_COLLEGE_WATER_POLO = 'water-polo/womens-college-water-polo'